In [1]:
from torch_geometric.data import DataLoader
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/data/conghao001/anaconda3/envs/digdrug/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
print(torch.version.cuda)

1.7.0
10.1


In [3]:
device = torch.device("cuda:2")
# device = torch.device("cpu")
device

device(type='cuda', index=2)

In [5]:
from utils import TestbedDataset
from models import GATNet_E, GATNet, GCNNet

In [6]:
model = GATNet()
model_path = 'root_folder/root_004/models/model_GAT-EP300-SW801010_GDSC.model'

# model = GCNNet()
# model_path = 'root_folder/root_003/models/model_GCN-EP300-SW801010_GDSC.model'
model.load_state_dict(torch.load(model_path, map_location=device))

RuntimeError: Error(s) in loading state_dict for GATNet:
	Missing key(s) in state_dict: "gcn1.att_l", "gcn1.att_r", "gcn1.lin_l.weight", "gcn1.lin_r.weight", "gcn2.att_l", "gcn2.att_r", "gcn2.lin_l.weight", "gcn2.lin_r.weight". 
	Unexpected key(s) in state_dict: "gcn1.att_src", "gcn1.att_dst", "gcn1.lin_src.weight", "gcn1.lin_dst.weight", "gcn2.att_src", "gcn2.att_dst", "gcn2.lin_src.weight", "gcn2.lin_dst.weight". 

In [ ]:
branch_folder = "root_folder/root_004"
dataset = 'GDSC'
test_data = TestbedDataset(root=branch_folder, dataset=dataset+'_test_mix')

In [ ]:
test_batch = 1
test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)

In [ ]:
import dig.xgraph.method as digmethod

In [12]:
import dig.xgraph

In [13]:
import dig.xgraph.method

ImportError: cannot import name 'ExpansionTypes' from 'captum.attr._utils.common' (/data/conghao001/anaconda3/envs/gnndrug/lib/python3.8/site-packages/captum/attr/_utils/common.py)